# Simulate synapses onto aCC

In [2]:
%matplotlib notebook

import sys
import csv

from neuron import h, gui
from neuron.units import ms, mV

from matplotlib import pyplot
import numpy as np
import pandas as pd

# allow importing module from parent folder
sys.path.append('../')

from synaptic_stimulation import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import Bokeh for graphs

In [3]:
from bokeh.io import output_notebook
import bokeh.plotting as plt
from bokeh.models.ranges import DataRange1d
from bokeh.models.tools import CrosshairTool, ExamineTool
from bokeh.io import export_svg

output_notebook()

Loading BokehJS ...

## Load morphology and other components

In [4]:
# load channels
h('''{load_file("../chan-DmNaT-ODowd.hoc")
      load_file("../chan-DmNaP-DmNav10.hoc")
      load_file("../chan-DmKdr-Marley.hoc")
      load_file("../chan-DmKA-Marley.hoc")
     }''')

# load morphology
h('{load_file("../neuron-morph-ext-axon-2pieces-chans-ext-axon-70um.hoc")}')

# enable or disable voltage clamp
h('load_file("../vclamp_soma_-60mV.ses")')

	0 
	0 
	0 
	0 
	0 
	1 


1

## Initialization

In [5]:
# load common funcs
h('load_file("../fitfuncs.hoc")')

#h('print_elec_cell()') #fix this 

# calc morph stats
h('load_file("../stats.hoc")')

file_name = "ext-axon-70um-sim-synapses"

# save state in this file
h.state_file_name = "state-neuron-act+elec+ext-axon-2piece-chans-" + file_name + ".bin"

# small adjustments

# increase VC duration
h.VClamp[0].dur[0] = 1000

	1 
Secname	L (um)	Diam (um)	Area [um^2]	Ri [MO]		Cm [pF]	g_pas [nS]
axon:	37.61	1.14	134.11	35.12	1.34	0.00
soma:	6.98	5.50	120.56	0.28	1.21	0.00
bottom dend:	996.91	0.78	2430.62	1991.49	24.31	0.02
botdend - axon:	959.10	0.76	2292.50	1993.51	22.92	0.02
topdend:	366.00	0.73	837.85	829.37	8.38	0.01
ext. axon:	570.00	0.71	1275.49	1351.81	12.75	1.53
	1 


In [6]:
'''
 * synapses:
 * top dend: 685, 524, 520, 626
 * bot dend: 205, 357, 464, 588, 513, 48
'''

create_syns(h, h.dendrite[685], h.dendrite[524], h.dendrite[520], h.dendrite[626], h.dendrite[205], h.dendrite[357], 
                h.dendrite[464], h.dendrite[588], h.dendrite[513], h.dendrite[48], # end of the 10 distal positions
                h.soma[0], h.dendrite[331], h.dendrite[271] # close to soma
                )
#set_syn_pars()                  # use defaults

Creating synapse on dendrite[685]
Creating synapse on dendrite[524]
Creating synapse on dendrite[520]
Creating synapse on dendrite[626]
Creating synapse on dendrite[205]
Creating synapse on dendrite[357]
Creating synapse on dendrite[464]
Creating synapse on dendrite[588]
Creating synapse on dendrite[513]
Creating synapse on dendrite[48]
Creating synapse on soma[0]
Creating synapse on dendrite[331]
Creating synapse on dendrite[271]


# Experiments

## create shape plot

In [7]:
ps = h.PlotShape(0) # don't show a default view
ps.show(0)         # show diameters
ps.view(30,10,55,87, 300, 100, 500, 700)
h.load_file('stdrun.hoc')

1.0

In [8]:
vc_current = h.Vector().record(h.VClamp[0]._ref_i)  # Voltage clamp current vector
v_soma = h.Vector().record(h.soma[0](0.5)._ref_v)  # Membrane potential vector
t = h.Vector().record(h._ref_t)  # Time stamp vector

## Run a short simulation to save steady state

In [8]:
# warning! if state changes (new synapses, etc) restoring saved state file will cause a crash!
h.finitialize(-60 * mV)
h.continuerun(100 * ms)
# rewind and save
h.finitialize()
h.saveState()
print("State saved to: " + h.state_file_name)

Saved state to file.
State saved to: state-neuron-act+elec+ext-axon-2piece-chans-ext-axon-70um-sim-synapses.bin


## Stimulate one synapse with varying magnitudes to check changes in rise and fall times that explain observed variability

Observed rise times vary between 0-9 ms and fall times vary between 0-30 ms.

In [9]:
syn = syns['dendrite[685]']

Synapses 685 is on the tip of the contralateral dendrite, but electrotonically not so far from the soma according to the 2015 paper passive/active parameters.

In [10]:
h.restoreState()
syn.set_stim(ps = ps, weight = .000001, taurise = 1, taufall=5, interval = 0, number = 1)
h.finitialize()
h.continuerun(60 * ms)

Reading state from file.
Setting: weight=1e-06, taurise = 1, taufall=5, interval=0, magepsc=20, syne=0, number=1, start=10


0.0

In [11]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="VC current (nA)")
f.add_tools(CrosshairTool())
f.line(np.array(t), np.array(vc_current), line_width=2)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -0.01, end = 0)
plt.show(f)

In [11]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-685.svg")

['single-input-dendrite-685.svg']

### Extract information

In [12]:
extract_mini_metrics(h, np.array(vc_current), 10)

{'risetime_ms': 3.200000000000001,
 'falltime_ms': 25.275000000000002,
 'amp_pA': -1.0387280346435546}

### Run a sweep of amplitudes

In [10]:
vc_currents_685 = run_amp_sweep(h, ps, vc_current, syn, .0001, 0.001, 6)

Simulating step of synaptic weight of 0.0001.
Reading state from file.
Setting: weight=0.0001, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00015848931924611142.
Reading state from file.
Setting: weight=0.00015848931924611142, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00025118864315095795.
Reading state from file.
Setting: weight=0.00025118864315095795, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00039810717055349735.
Reading state from file.
Setting: weight=0.00039810717055349735, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.000630957344480193.
Reading state from file.
Setting: weight=0.000630957344480193, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic wei

In [70]:
# no need with autoreload?
import importlib
importlib.reload(sys.modules['synaptic_stimulation'])

<module 'synaptic_stimulation' from '/home/cengiz/drosophila/drosophila-aCC-L3-motoneuron-model/neuron-model/tutorial-synapse-activation/../synaptic_stimulation.py'>

In [11]:
plot_amp_sweep(t, vc_currents_685)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-685-amp-sweep.svg")

In [12]:
minis_den685_df = get_sweep_metrics(h, vc_currents_685)
plot_scatter_risetime_amp(minis_den685_df)
minis_den685_df

,risetime_ms,falltime_ms,amp_pA
0,5.700,39.600,-5.546413
1,5.700,41.975,-8.701848
2,5.700,44.325,-13.574619
3,5.700,44.325,-20.991027
4,5.700,44.325,-32.033066
5,5.675,44.350,-47.945314


Although fall time increased with varying amplitudes, there was no discernible change in the rise time. So rise time is not affected by stimulus amplitude (find refs to back this up). 685 is on the contralateral dendrite and closer to the soma, so we expect more delays from the ipsilateral dendrite.

## Second synapse

In [13]:
syn2 = syns['dendrite[588]']

In [21]:
#syn.set_stim(ps = ps, weight = 0, tau=5, interval = 0, number = 1)
h.restoreState()
#syn.set_stim(ps = ps, weight = .0018, tau=5, interval = 0, number = 1)
syn2.set_stim(ps = ps, weight = .00018, taurise=1, taufall=5, interval = 0, number = 1)
h.finitialize()
h.continuerun(60 * ms)

Reading state from file.
Setting: weight=0.00018, taurise = 1, taufall=5, interval=0, magepsc=20, syne=0, number=1, start=10


0.0

In [22]:
f = plt.figure(x_axis_label="t (ms)", y_axis_label="VC current (nA)")
f.add_tools(CrosshairTool())
f.line(np.array(t), np.array(vc_current), line_width=2)
f.x_range = DataRange1d(start = 9, end = 50)
f.y_range = DataRange1d(start = -0.10, end = 0)
plt.show(f)

In [14]:
vc_currents_588 = run_amp_sweep(h, ps, vc_current, syn2, .0001, 0.001, 6)

Simulating step of synaptic weight of 0.0001.
Reading state from file.
Setting: weight=0.0001, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00015848931924611142.
Reading state from file.
Setting: weight=0.00015848931924611142, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00025118864315095795.
Reading state from file.
Setting: weight=0.00025118864315095795, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00039810717055349735.
Reading state from file.
Setting: weight=0.00039810717055349735, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.000630957344480193.
Reading state from file.
Setting: weight=0.000630957344480193, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic wei

In [33]:
plot_amp_sweep(t, vc_currents_588)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-588-amp-sweep.svg")

In [15]:
minis_den588_df = get_sweep_metrics(h, vc_currents_588)
plot_scatter_risetime_amp(minis_den588_df)
minis_den588_df

,risetime_ms,falltime_ms,amp_pA
0,6.825,39.700,-5.187911
1,6.825,42.075,-8.146395
2,6.825,43.200,-12.724929
3,6.800,43.225,-19.716490
4,6.775,43.250,-30.176708
5,6.750,43.275,-45.355139


Same issue as 685: rise time does not change with stimulus amplitude. Increasing synapse $\tau_{rise}$ parameter also inreases observed rise time, but does not increase its variance.

## Ipsilateral, central synapse

In [20]:
syn3 = syns['dendrite[48]']

In [21]:
vc_currents_48 = run_amp_sweep(h, ps, vc_current, syn3, .0001, 0.002, 6)

Simulating step of synaptic weight of 0.0001.
Reading state from file.
Setting: weight=0.0001, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00018205642030260795.
Reading state from file.
Setting: weight=0.00018205642030260795, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00033144540173399875.
Reading state from file.
Setting: weight=0.00033144540173399875, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.0006034176336545162.
Reading state from file.
Setting: weight=0.0006034176336545162, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.0010985605433061184.
Reading state from file.
Setting: weight=0.0010985605433061184, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic wei

In [22]:
plot_amp_sweep(t, vc_currents_48)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-48-amp-sweep.svg")

In [23]:
minis_den48_df = get_sweep_metrics(h, vc_currents_48)
plot_scatter_risetime_amp(minis_den48_df)
minis_den48_df

,risetime_ms,falltime_ms,amp_pA
0,6.625,39.675,-5.206325
1,6.600,42.800,-9.393004
2,6.600,43.425,-16.824425
3,6.575,43.450,-29.754121
4,6.550,43.475,-51.483457
5,6.500,43.525,-85.945957


## Synapse at soma

In [16]:
syn4 = syns['soma[0]']

In [17]:
vc_currents_soma = run_amp_sweep(h, ps, vc_current, syn4, .0001, 0.001, 6)

Simulating step of synaptic weight of 0.0001.
Reading state from file.
Setting: weight=0.0001, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00015848931924611142.
Reading state from file.
Setting: weight=0.00015848931924611142, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00025118864315095795.
Reading state from file.
Setting: weight=0.00025118864315095795, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.00039810717055349735.
Reading state from file.
Setting: weight=0.00039810717055349735, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic weight of 0.000630957344480193.
Reading state from file.
Setting: weight=0.000630957344480193, taurise = 4, taufall=5, interval=0, magepsc=1, syne=0, number=1, start=10
Simulating step of synaptic wei

In [37]:
plot_amp_sweep(t, vc_currents_soma)

In [ ]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="single-input-dendrite-48-amp-sweep.svg")

In [18]:
minis_soma0_df = get_sweep_metrics(h, vc_currents_soma)
plot_scatter_risetime_amp(minis_soma0_df)
minis_soma0_df

,risetime_ms,falltime_ms,amp_pA
0,5.2,39.625,-5.659504
1,5.2,42.000,-8.958495
2,5.2,44.375,-14.170264
3,5.2,44.825,-22.388476
4,5.2,44.825,-35.309273
5,5.2,44.825,-55.529656


# Compare metrics

In [24]:
f = plt.figure(x_axis_label="rise time (ms)", y_axis_label="amplitude (pA)", frame_width = 400, frame_height = 400)
f.add_tools(CrosshairTool())
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_soma0_df, color='purple', legend_label='soma0')
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den685_df, color='red', legend_label='dend685')
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den48_df, color='green', legend_label='dend48')
f.scatter(x='risetime_ms', y='amp_pA',  source=minis_den588_df, color='blue', legend_label='dend588')
#f.x_range = DataRange1d(start = 0.8, end = 1.1)
#f.y_range = DataRange1d(start = -70, end = 0)
f.legend.location = 'bottom_left'
plt.show(f)

In [48]:
# save as SVG
f.output_backend = "svg"
export_svg(f, filename="compare-scatter-amp-sweeps.svg")

['compare-scatter-amp-sweeps.svg']

## To Do
- check if changing passive parameters affect rise time variance
- make plot of distance while keeping amplitude constant

In [ ]:
np.linspace(0.0001, 0.1, 10)

In [34]:
syns

{}